In [1]:
from dotenv import load_dotenv
import os

load_dotenv("../../.env")

True

In [2]:
from cdisc_library import CDISCConnector

conn = CDISCConnector(os.environ['CDISC_USER'], os.environ['CDISC_PASS'])


In [3]:
prods = conn.products

In [4]:
print(prods)

{'cdash': '/mdr/cdash/1-0', 'cdashig': '/mdr/cdashig/1-1-1', 'sdtm': '/mdr/sdtm/1-2', 'sdtmig': '/mdr/sdtmig/3-1-2', 'sendig': '/mdr/sendig/3-0', 'adam': '/mdr/adam/adam-adae-1-0', 'packages': '/mdr/ct/packages/adamct-2014-09-26'}


In [5]:
from podr_connections import podr_connection

In [6]:
cc = podr_connection(os.environ["PODR_USER"], os.environ["PODR_PASS"])


In [38]:
DM = cc.read("DM", "janssen_synthetic")

EX = cc.read("EX", "janssen_synthetic")

DS = cc.read("DS", "janssen_synthetic")

In [8]:
import re
patt = re.compile("([A-Z]{2})\.([A-Z]+)")


In [9]:
patt.match("EX.EXSTDTC").groups()



('EX', 'EXSTDTC')

In [10]:
sample = """Subjects who are randomly assigned to a treatment group and have a baseline measurement of body weight.  
NOTE:  For baseline weight select VS.VSSTRESN value where VS.VSTESTCD=WEIGHT and VSDT <= ARFSTDT, consider the last non missing value"""

In [11]:
patt.findall(sample)

[('VS', 'VSSTRESN'), ('VS', 'VSTESTCD')]

In [12]:
sample_two ="""Date of the initial study agent administration (EX.EXSTDTC), or randomization date (DS.DSSTDTC where DS.DSDECOD  equal to  'RANDOMIZED) if subject was not dosed. Convert to numeric date.
Note: For Treatment A dose is 0 which is considered as dosed"""


In [13]:
patt.findall(sample_two)

[('EX', 'EXSTDTC'), ('DS', 'DSSTDTC'), ('DS', 'DSDECOD')]

In [14]:
from yaml import load


In [16]:
with open("adsl.yaml", 'r') as fh:
    config = load(fh)


/opt/python/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [32]:
config

{'TRT01P': {'source': 'DM.ARM'},
 'TRT01PN': {'dependencies': ['TRT01P'], 'logic': None},
 'TRT01A': {'source': 'DM.ACTARM'},
 'TRT01AN': {'dependencies': ['TRT01A']},
 'RANDFL': {'dependencies': ['DS.USUBJID', 'DS.DSDECOD'],
  'logic': '"Y" if [ ( DS["USUBJID"] == ADSL["USUBJID"] ) & ( DS["DSDECOD"] == "RANDOMIZED" ) ] else "N"\n'},
 'ITTFL': {'dependencies': ['ARFSTDT'],
  'logic': '"Y" if [ ( VS["USUBJID"] == ADSL["USUBJID"] ) & ( VS["VSDT"] <= ADSL["ARFSTDT"] ) ] else "N"\n'},
 'ARFSTDT': {'logic': ['notnull',
   '[ ( EX["USUBJID"] == ADSL["USUBJID"] ) & ( VS["VSDT"] <= ADSL["ARFSTDT"] ) ]\n',
   '[ ( EX["USUBJID"] == ADSL["USUBJID"] ) & ( DS["DSDECOD"] == "RANDOMIZED" ) ]\n']}}

In [8]:
DM["DTHDTC"]


0      None
1      None
2      None
3      None
4      None
       ... 
104    None
105    None
106    None
107    None
108    None
Name: DTHDTC, Length: 109, dtype: object

In [39]:
TA = cc.read("TA", "janssen_synthetic")

In [40]:
VS = cc.read("VS", "janssen_synthetic")

In [41]:
DS = cc.read("DS", "janssen_synthetic")

In [45]:
st_ex = EX.groupby(["STUDYID", "USUBJID"]).agg({"EXSTDTC": 'min'})

In [46]:
st_ex


EXSTDTC
STUDYID             USUBJID                                        
64565111OBE2001-syn 64565111OBE2001-syn-9900001 2018-10-28 11:10:00
                    64565111OBE2001-syn-9900005 2018-07-26 10:21:00
                    64565111OBE2001-syn-9900014 2018-09-03 09:27:00
                    64565111OBE2001-syn-9900016 2018-12-19 11:15:00
                    64565111OBE2001-syn-9900031 2018-09-14 12:30:00
...                                                             ...
64565111OBE2002-syn 64565111OBE2002-syn-9900280 2018-12-20 00:00:00
                    64565111OBE2002-syn-9900287 2018-08-04 00:00:00
                    64565111OBE2002-syn-9900300 2018-10-23 00:00:00
                    64565111OBE2002-syn-9900309 2019-01-14 00:00:00
                    64565111OBE2002-syn-9900313 2018-04-16 00:00:00

[109 rows x 1 columns]

In [47]:
ds_rnd = DS[DS["DSDECOD"] == "RANDOMIZED"].groupby(["STUDYID", "USUBJID"]).agg({"DSSTDTC": "min"})

In [48]:
ds_rnd

DSSTDTC
STUDYID             USUBJID                               
64565111OBE2001-syn 64565111OBE2001-syn-9900001 2018-10-28
                    64565111OBE2001-syn-9900005 2018-07-26
                    64565111OBE2001-syn-9900014 2018-09-03
                    64565111OBE2001-syn-9900016 2018-12-19
                    64565111OBE2001-syn-9900031 2018-09-14
...                                                    ...
64565111OBE2002-syn 64565111OBE2002-syn-9900280 2018-12-20
                    64565111OBE2002-syn-9900287 2018-08-04
                    64565111OBE2002-syn-9900300 2018-10-23
                    64565111OBE2002-syn-9900309 2019-01-14
                    64565111OBE2002-syn-9900313 2018-04-16

[109 rows x 1 columns]

In [49]:
import pandas as pd


In [51]:
cc = pd.concat([st_ex, ds_rnd], keys=["STUDYID", "USUBJID"])

In [52]:
cc

EXSTDTC  \
        STUDYID             USUBJID                                           
STUDYID 64565111OBE2001-syn 64565111OBE2001-syn-9900001 2018-10-28 11:10:00   
                            64565111OBE2001-syn-9900005 2018-07-26 10:21:00   
                            64565111OBE2001-syn-9900014 2018-09-03 09:27:00   
                            64565111OBE2001-syn-9900016 2018-12-19 11:15:00   
                            64565111OBE2001-syn-9900031 2018-09-14 12:30:00   
...                                                                     ...   
USUBJID 64565111OBE2002-syn 64565111OBE2002-syn-9900280                 NaT   
                            64565111OBE2002-syn-9900287                 NaT   
                            64565111OBE2002-syn-9900300                 NaT   
                            64565111OBE2002-syn-9900309                 NaT   
                            64565111OBE2002-syn-9900313                 NaT   

                                                           DSSTDTC  
        STUDYID             USUBJID                                 
STUDYID 64565111OBE2001-syn 64565111OBE2001-syn-9900001        NaT  
                            64565111OBE2001-syn-9900005        NaT  
                            64565111OBE2001-syn-9900014        NaT  
                            64565111OBE2001-syn-9900016        NaT  
                            64565111OBE2001-syn-9900031        NaT  
...                                                            ...  
USUBJID 64565111OBE2002-syn 64565111OBE2002-syn-9900280 2018-12-20  
                            64565111OBE2002-syn-9900287 2018-08-04  
                            64565111OBE2002-syn-9900300 2018-10-23  
                            64565111OBE2002-syn-9900309 2019-01-14  
                            64565111OBE2002-syn-9900313 2018-04-16  

[218 rows x 2 columns]

In [44]:
st_ex


,STUDYID,DOMAIN,USUBJID,EXSEQ,EXSPID,EXLNKID,EXTRT,EXCAT,EXDOSE,EXDOSU,EXDOSFRM,EXDOSFRQ,EXROUTE,EPOCH,EXSTDTC,EXENDTC,EXSTDY,EXENDY
1347,64565111OBE2001-syn,EX,64565111OBE2001-syn-9900523,1.0,None,************,TREATMENT A,TREATMENT A,0.0,mg,SC INJECTION,1 TIME PER WEEK,None,TREATMENT,2017-12-20 12:00:00,2017-12-20 12:00:00,1.0,1.0


In [35]:
source_dm = ["STUDYID", "USUBJID", "ARM", "ARMCD", "ACTARM", "ACTARMCD", "AGE", "AGEU", "SEX", "RACE", "COUNTRY", "DTHDTC", "DTHFL", "INVNAM", "SITEID", "SUBJID"]

In [36]:
ADSL = DM[source_dm].copy()


In [37]:
ADSL["RANDFL"] = ADSL.apply(lambda ADSL: "Y" if [ (DS["USUBJID"] == ADSL["USUBJID"]) & (DS["DSDECOD"] == "RANDOMIZED") ] else "N", axis=1)


NameError: name 'DS' is not defined

In [75]:
"""
Date of the initial study agent administration (EX.EXSTDTC), or randomization date (DS.DSSTDTC where DS.DSDECOD  equal to 'RANDOMIZED) if subject was not dosed. Convert to numeric date.
Note: For Treatment A dose is 0 which is considered as dosed
"""
rand_st = ADSL.apply(lambda x: DS[(DS["DSDECOD"] == "RANDOMIZED") & (DS["USUBJID"] == x["USUBJID"])]["DSSTDTC"], axis=1)


In [77]:
EX


,STUDYID,DOMAIN,USUBJID,EXSEQ,EXSPID,EXLNKID,EXTRT,EXCAT,EXDOSE,EXDOSU,EXDOSFRM,EXDOSFRQ,EXROUTE,EPOCH,EXSTDTC,EXENDTC,EXSTDY,EXENDY
0,64565111OBE2001-syn,EX,64565111OBE2001-syn-9900001,1.0,None,************,TREATMENT A,TREATMENT A,0.0,mg,SC INJECTION,1 TIME PER WEEK,None,TREATMENT,2018-10-28 11:10:00,2018-10-28 11:10:00,1.0,1.0
1,64565111OBE2001-syn,EX,64565111OBE2001-syn-9900001,2.0,None,************,TREATMENT A,TREATMENT A,0.0,mg,SC INJECTION,1 TIME PER WEEK,None,TREATMENT,2018-11-04 10:40:00,2018-11-04 10:40:00,8.0,8.0
2,64565111OBE2001-syn,EX,64565111OBE2001-syn-9900001,3.0,None,************,TREATMENT A,TREATMENT A,0.0,mg,SC INJECTION,1 TIME PER WEEK,None,TREATMENT,2018-11-11 11:40:00,2018-11-11 11:40:00,15.0,15.0
3,64565111OBE2001-syn,EX,64565111OBE2001-syn-9900001,4.0,None,************,TREATMENT A,TREATMENT A,0.0,mg,SC INJECTION,1 TIME PER WEEK,None,TREATMENT,2018-11-18 11:00:00,2018-11-18 11:00:00,22.0,22.0
4,64565111OBE2001-syn,EX,64565111OBE2001-syn-9900001,5.0,None,************,TREATMENT A,TREATMENT A,0.0,mg,SC INJECTION,1 TIME PER WEEK,None,TREATMENT,2018-11-25 11:45:00,2018-11-25 11:45:00,29.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,64565111OBE2002-syn,EX,64565111OBE2002-syn-9900313,9.0,None,****************,TREATMENT B,TREATMENT B,10.0,mg,INJECTION,1 TIME PER WEEK,SUBCUTANEOUS,TREATMENT,2018-06-11 00:00:00,2018-06-11 00:00:00,57.0,57.0
2012,64565111OBE2002-syn,EX,64565111OBE2002-syn-9900313,10.0,None,****************,TREATMENT B,TREATMENT B,10.0,mg,INJECTION,1 TIME PER WEEK,SUBCUTANEOUS,TREATMENT,2018-06-18 00:00:00,2018-06-18 00:00:00,64.0,64.0
2013,64565111OBE2002-syn,EX,64565111OBE2002-syn-9900313,11.0,None,****************,TREATMENT B,TREATMENT B,10.0,mg,INJECTION,1 TIME PER WEEK,SUBCUTANEOUS,TREATMENT,2018-06-25 00:00:00,2018-06-25 00:00:00,71.0,71.0
2014,64565111OBE2002-syn,EX,64565111OBE2002-syn-9900313,12.0,None,****************,TREATMENT B,TREATMENT B,10.0,mg,INJECTION,1 TIME PER WEEK,SUBCUTANEOUS,TREATMENT,2018-07-02 00:00:00,2018-07-02 00:00:00,78.0,78.0


In [84]:
exstdtc = EX.loc[EX.groupby("USUBJID").EXSTDTC.idxmin()]["EXSTDTC"]

In [85]:
exstdtc

0      2018-10-28 11:10:00
26     2018-07-26 10:21:00
53     2018-09-03 09:27:00
80     2018-12-19 11:15:00
90     2018-09-14 12:30:00
               ...        
1954   2018-12-20 00:00:00
1966   2018-08-04 00:00:00
1979   2018-10-23 00:00:00
1990   2019-01-14 00:00:00
2003   2018-04-16 00:00:00
Name: EXSTDTC, Length: 109, dtype: datetime64[ns]

In [80]:
exstdtc


,STUDYID,DOMAIN,USUBJID,EXSEQ,EXSPID,EXLNKID,EXTRT,EXCAT,EXDOSE,EXDOSU,EXDOSFRM,EXDOSFRQ,EXROUTE,EPOCH,EXSTDTC,EXENDTC,EXSTDY,EXENDY
0,64565111OBE2001-syn,EX,64565111OBE2001-syn-9900001,1.0,None,************,TREATMENT A,TREATMENT A,0.0,mg,SC INJECTION,1 TIME PER WEEK,None,TREATMENT,2018-10-28 11:10:00,2018-10-28 11:10:00,1.0,1.0
26,64565111OBE2001-syn,EX,64565111OBE2001-syn-9900005,1.0,None,************,TREATMENT A,TREATMENT A,0.0,mg,SC INJECTION,1 TIME PER WEEK,None,TREATMENT,2018-07-26 10:21:00,2018-07-26 10:21:00,1.0,1.0
53,64565111OBE2001-syn,EX,64565111OBE2001-syn-9900014,1.0,None,************,TREATMENT A,TREATMENT A,0.0,mg,SC INJECTION,1 TIME PER WEEK,None,TREATMENT,2018-09-03 09:27:00,2018-09-03 09:27:00,1.0,1.0
80,64565111OBE2001-syn,EX,64565111OBE2001-syn-9900016,1.0,None,************,TREATMENT A,TREATMENT A,0.0,mg,SC INJECTION,1 TIME PER WEEK,None,TREATMENT,2018-12-19 11:15:00,2018-12-19 11:15:00,1.0,1.0
90,64565111OBE2001-syn,EX,64565111OBE2001-syn-9900031,1.0,None,************,TREATMENT A,TREATMENT A,0.0,mg,SC INJECTION,1 TIME PER WEEK,None,TREATMENT,2018-09-14 12:30:00,2018-09-14 12:30:00,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1954,64565111OBE2002-syn,EX,64565111OBE2002-syn-9900280,1.0,None,****************,TREATMENT B,TREATMENT B,10.0,mg,INJECTION,1 TIME PER WEEK,SUBCUTANEOUS,TREATMENT,2018-12-20 00:00:00,2018-12-20 00:00:00,1.0,1.0
1966,64565111OBE2002-syn,EX,64565111OBE2002-syn-9900287,1.0,None,****************,TREATMENT B,TREATMENT B,7.4,mg,INJECTION,1 TIME PER WEEK,SUBCUTANEOUS,TREATMENT,2018-08-04 00:00:00,2018-08-04 00:00:00,1.0,1.0
1979,64565111OBE2002-syn,EX,64565111OBE2002-syn-9900300,1.0,None,****************,TREATMENT B,TREATMENT B,10.0,mg,INJECTION,1 TIME PER WEEK,SUBCUTANEOUS,TREATMENT,2018-10-23 00:00:00,2018-10-23 00:00:00,1.0,1.0
1990,64565111OBE2002-syn,EX,64565111OBE2002-syn-9900309,1.0,None,****************,TREATMENT B,TREATMENT B,5.0,mg,INJECTION,1 TIME PER WEEK,SUBCUTANEOUS,TREATMENT,2019-01-14 00:00:00,2019-01-14 00:00:00,1.0,1.0


In [76]:
rand_st

,4,9,14,19,24,29,34,39,44,49,...,508,513,518,523,528,533,538,543,548,553
0,2018-10-28,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,NaT,2018-07-26,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,NaT,NaT,2018-09-03,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,NaT,NaT,NaT,2018-12-19,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,NaT,NaT,NaT,NaT,2018-09-14,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,2018-12-20,NaT,NaT,NaT,NaT
105,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,2018-08-04,NaT,NaT,NaT
106,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2018-10-23,NaT,NaT
107,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-01-14,NaT


In [71]:
rand_st

0    4     2018-10-28
1    9     2018-07-26
2    14    2018-09-03
3    19    2018-12-19
4    24    2018-09-14
              ...    
104  533   2018-12-20
105  538   2018-08-04
106  543   2018-10-23
107  548   2019-01-14
108  553   2018-04-16
Length: 109, dtype: datetime64[ns]

In [68]:
import pandas
ADSL = ADSL.concat(ADSL, rand_st)

AttributeError: 'DataFrame' object has no attribute 'concat'

In [44]:
DS.query("DSDECOD == 'RANDOMIZED' and USUBJID == '64565111OBE2001-syn-9900001'")["DSSTDTC"]

4   2018-10-28
Name: DSSTDTC, dtype: datetime64[ns]

In [30]:
import datetime
import pandas as pd

subjid = '64565111OBE2001-syn-9900001'
"Y" if [(VS["USUBJID"] == subjid) 
        & (VS["VSTESTCD"] == "WEIGHT") 
        & (VS["VSDTC"] <= datetime.date.today()) ] else "N"

AssertionError: 

In [1]:
import re
re.match("(\d{4})-(\d{2})-(\d{2})", "2019-12-12")

<re.Match object; span=(0, 10), match='2019-12-12'>

In [3]:
import pandas as pd
pd.to_datetime("2019-12-12", format="%Y-%m-%d")

Timestamp('2019-12-12 00:00:00')

In [15]:
VS.dtypes

STUDYID      object
DOMAIN       object
USUBJID      object
VSSEQ       float64
VSGRPID      object
VSSPID       object
VSLNKID      object
VSTESTCD     object
VSTEST       object
VSCAT        object
VSPOS        object
VSORRES      object
VSORRESU     object
VSSTRESC     object
VSSTRESN    float64
VSSTRESU     object
VSSTAT       object
VSREASND     object
VSBLFL       object
VISITNUM    float64
VISIT        object
VISITDY     float64
EPOCH        object
VSDTC        object
VSDY        float64
dtype: object

In [18]:
type(VS["VSDTC"][0])

str

In [ ]:
s

In [8]:
len(dm)

109

In [11]:
dm[dm["RACE"] != "WHITE" ]

,STUDYID,DOMAIN,USUBJID,SUBJID,RFSTDTC,RFENDTC,RFXSTDTC,RFXENDTC,RFICDTC,RFPENDTC,...,SEX,RACE,ETHNIC,ARMCD,ARM,ACTARMCD,ACTARM,COUNTRY,DMDTC,DMDY
12,64565111OBE2001-syn,DM,64565111OBE2001-syn-9900115,9900115,2018-06-11T11:50,2019-01-07,2018-06-11T11:50,2018-12-03T10:30,2018-05-20,2019-01-07T10:30:00,...,F,ASIAN,NOT HISPANIC OR LATINO,TRTA,TREATMENT A,TRTA,TREATMENT A,CAN,2018-05-20,-22.0
16,64565111OBE2001-syn,DM,64565111OBE2001-syn-9900137,9900137,2018-03-23T08:50,2018-10-26,2018-03-23T08:50,2018-09-10T16:00,2018-03-04,2018-10-26T13:15:00,...,M,MULTIPLE,NOT HISPANIC OR LATINO,TRTA,TREATMENT A,TRTA,TREATMENT A,GBR,2018-03-04,-19.0
35,64565111OBE2001-syn,DM,64565111OBE2001-syn-9900319,9900319,2018-08-24T11:05,2019-03-21,2018-08-24T11:05,2019-02-08T10:30,2018-08-02,2019-03-21T09:40:00,...,F,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,TRTA,TREATMENT A,TRTA,TREATMENT A,SWE,2018-08-02,-22.0
40,64565111OBE2001-syn,DM,64565111OBE2001-syn-9900376,9900376,2019-01-29T10:16,2019-08-17,2019-01-29T10:16,2019-07-09T05:45,2018-12-29,2019-08-17T08:06:00,...,M,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,TRTA,TREATMENT A,TRTA,TREATMENT A,USA,2018-12-29,-31.0
44,64565111OBE2001-syn,DM,64565111OBE2001-syn-9900420,9900420,2018-10-25T07:35,2019-05-21,2018-10-25T07:35,2019-04-18T12:30,2018-10-03,2019-05-21T15:00:00,...,F,MULTIPLE,NOT HISPANIC OR LATINO,TRTA,TREATMENT A,TRTA,TREATMENT A,USA,2018-10-03,-22.0
46,64565111OBE2001-syn,DM,64565111OBE2001-syn-9900470,9900470,2018-09-28T08:12,2019-05-03,2018-09-28T08:12,2019-03-29T06:30,2018-09-06,2019-05-03T09:31:00,...,M,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,TRTA,TREATMENT A,TRTA,TREATMENT A,USA,2018-09-06,-22.0
53,64565111OBE2001-syn,DM,64565111OBE2001-syn-9900501,9900501,2019-01-21T11:42,2019-08-12,2019-01-21T11:42,2019-07-08T22:33,2019-01-01,2019-08-12T08:25:00,...,F,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,TRTA,TREATMENT A,TRTA,TREATMENT A,USA,2019-01-01,-20.0
55,64565111OBE2001-syn,DM,64565111OBE2001-syn-9900520,9900520,2018-04-08T14:58,2018-11-06,2018-04-08T14:58,2018-09-30T18:00,2018-03-18,2018-11-06T16:00:00,...,F,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,TRTA,TREATMENT A,TRTA,TREATMENT A,USA,2018-03-18,-21.0
56,64565111OBE2001-syn,DM,64565111OBE2001-syn-9900523,9900523,2017-12-20T12:00,2018-06-17,2017-12-20T12:00,2018-03-21T16:05,2017-11-29,2018-06-17T09:09,...,F,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,TRTA,TREATMENT A,TRTA,TREATMENT A,USA,2017-11-29,-21.0
59,64565111OBE2001-syn,DM,64565111OBE2001-syn-9900566,9900566,2018-04-30T13:00,2018-07-09,2018-04-30T13:00,2018-07-09T10:00,2018-03-24,2018-07-09T12:11:00,...,F,None,NOT HISPANIC OR LATINO,TRTA,TREATMENT A,TRTA,TREATMENT A,USA,2018-03-24,-37.0
